In [ ]:
import os
import json
import google.oauth2.credentials
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow
from datetime import datetime, timedelta

# YouTube analytics configuration

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/yt-analytics.readonly"]
API_SERVICE_NAME = "youtubeAnalytics"
API_VERSION = "v2"

Before running this sample locally for the first time, you need to [set up authorization credentials](https://developers.google.com/youtube/reporting/v1/code_samples/python#set-up-authorization-credentials) for your project.

Your `CLIENT_SECRETS_FILE` variable specifies the name of a file that contains the OAuth 2.0 information for this application, including its client_id and client_secret.

In [ ]:
CLIENT_SECRETS_FILE = "client_secret.json"

Disable OAuthlib's HTTPs verification when running locally. *DO NOT* leave this option enabled when running in production.

In [ ]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

Authorize the request and store authorization credentials.

In [ ]:
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
credentials = flow.run_console()
youtubeAnalytics = build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

# Request channel data

We want data from the period starting from the beginning of the current year, up until today.

In [ ]:
today = datetime.now().today()
start_date = today.strftime("%Y-01-01")
end_date = today.strftime("%Y-%m-%d")

Make the request for the following metrics: estimatedMinutesWatched, views, likes, subscribersGained.

In [ ]:
response = (
    youtubeAnalytics.reports()
    .query(
        ids="channel==MINE",
        startDate=start_date,
        endDate=end_date,
        metrics="estimatedMinutesWatched,views,likes,subscribersGained",
        dimensions="day",
        sort="day",
    )
    .execute()
)

Display column headers in retrieved data

In [ ]:
response["columnHeaders"]

Peek at the last 5 rows

In [ ]:
response["rows"][-5:]

Convert `estimatedMinutesWatched` from minutes to hours

In [ ]:
for row in response["rows"]:
    row[1] = round(row[1] / 60, 2)

# Export data for reporting

In [ ]:
youtube_analytics_data = {
    "rows": response["rows"],
    "columns": ["Date", "Watch time (hours)", "Views", "Likes", "Subscribers"],
}

with open("youtube_analytics_data.json", "w") as f:
    json.dump(youtube_analytics_data, f)